In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=ddeb93cdc9619101ecc15664220d5233d57cc29ec43c69215850bb9b996b9d4b
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("tutorial-2").getOrCreate()

In [4]:
!curl https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/main/Dataset/sales_data.csv >> sales_data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   202  100   202    0     0   1286      0 --:--:-- --:--:-- --:--:--  1286


In [5]:
df = spark.read.csv("/content/sales_data.csv", header= True, inferSchema= True)

In [6]:
df.show()

+-------+--------------+---------+
|company|representative|num_sales|
+-------+--------------+---------+
|    XYZ|           Bob|    200.0|
|    XYZ|           Tom|    120.0|
|    XYZ|         Frank|    340.0|
|   ABCD|         Jerry|    600.0|
|   ABCD|           Amy|    124.0|
|   ABCD|       Vanessa|    243.0|
|     OK|          Carl|    870.0|
|     OK|         Sarah|    350.0|
|   BLAH|          John|    250.0|
|   BLAH|         Linda|    130.0|
|   BLAH|          Mike|    750.0|
|   BLAH|         Chris|    350.0|
+-------+--------------+---------+



In [7]:
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- representative: string (nullable = true)
 |-- num_sales: double (nullable = true)



In [10]:
df.groupBy("company")

In [11]:
df.groupBy("company").mean().show()

+-------+-----------------+
|company|   avg(num_sales)|
+-------+-----------------+
|   BLAH|            370.0|
|    XYZ|            220.0|
|     OK|            610.0|
|   ABCD|322.3333333333333|
+-------+-----------------+



In [13]:
df.groupBy("company").count().show()

+-------+-----+
|company|count|
+-------+-----+
|   BLAH|    4|
|    XYZ|    3|
|     OK|    2|
|   ABCD|    3|
+-------+-----+



In [14]:
df.groupBy("company").min().show()

+-------+--------------+
|company|min(num_sales)|
+-------+--------------+
|   BLAH|         130.0|
|    XYZ|         120.0|
|     OK|         350.0|
|   ABCD|         124.0|
+-------+--------------+



In [15]:
df.groupBy("company").max().show()

+-------+--------------+
|company|max(num_sales)|
+-------+--------------+
|   BLAH|         750.0|
|    XYZ|         340.0|
|     OK|         870.0|
|   ABCD|         600.0|
+-------+--------------+



In [16]:
df.groupBy("company").sum().show()

+-------+--------------+
|company|sum(num_sales)|
+-------+--------------+
|   BLAH|        1480.0|
|    XYZ|         660.0|
|     OK|        1220.0|
|   ABCD|         967.0|
+-------+--------------+



In [17]:
df.agg({'num_sales': 'max'}).show()

+--------------+
|max(num_sales)|
+--------------+
|         870.0|
+--------------+



In [18]:
df.groupBy('company').agg({'num_sales': 'mean'}).show()

+-------+-----------------+
|company|   avg(num_sales)|
+-------+-----------------+
|   BLAH|            370.0|
|    XYZ|            220.0|
|     OK|            610.0|
|   ABCD|322.3333333333333|
+-------+-----------------+



In [19]:
company_groups= df.groupBy('company')

In [20]:
company_groups.min().show()

+-------+--------------+
|company|min(num_sales)|
+-------+--------------+
|   BLAH|         130.0|
|    XYZ|         120.0|
|     OK|         350.0|
|   ABCD|         124.0|
+-------+--------------+



In [21]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [22]:
df.select(countDistinct("num_sales")).show()

+-------------------------+
|count(DISTINCT num_sales)|
+-------------------------+
|                       11|
+-------------------------+



In [23]:
df.select(avg("num_sales")).show()

+-----------------+
|   avg(num_sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [24]:
df.select(stddev("num_sales")).show()

+----------------------+
|stddev_samp(num_sales)|
+----------------------+
|    250.08742410799007|
+----------------------+



In [25]:
from pyspark.sql.functions import format_number

In [26]:
sales_std= df.select(stddev('num_sales').alias('stddev'))

In [27]:
sales_std.show()

+------------------+
|            stddev|
+------------------+
|250.08742410799007|
+------------------+



In [28]:
sales_std.select(format_number("stddev", 2)).show()

+------------------------+
|format_number(stddev, 2)|
+------------------------+
|                  250.09|
+------------------------+



In [29]:
df.orderBy("num_sales").show()

+-------+--------------+---------+
|company|representative|num_sales|
+-------+--------------+---------+
|    XYZ|           Tom|    120.0|
|   ABCD|           Amy|    124.0|
|   BLAH|         Linda|    130.0|
|    XYZ|           Bob|    200.0|
|   ABCD|       Vanessa|    243.0|
|   BLAH|          John|    250.0|
|    XYZ|         Frank|    340.0|
|     OK|         Sarah|    350.0|
|   BLAH|         Chris|    350.0|
|   ABCD|         Jerry|    600.0|
|   BLAH|          Mike|    750.0|
|     OK|          Carl|    870.0|
+-------+--------------+---------+



In [30]:
df.orderBy(df["num_sales"].desc()).show()

+-------+--------------+---------+
|company|representative|num_sales|
+-------+--------------+---------+
|     OK|          Carl|    870.0|
|   BLAH|          Mike|    750.0|
|   ABCD|         Jerry|    600.0|
|     OK|         Sarah|    350.0|
|   BLAH|         Chris|    350.0|
|    XYZ|         Frank|    340.0|
|   BLAH|          John|    250.0|
|   ABCD|       Vanessa|    243.0|
|    XYZ|           Bob|    200.0|
|   BLAH|         Linda|    130.0|
|   ABCD|           Amy|    124.0|
|    XYZ|           Tom|    120.0|
+-------+--------------+---------+



In [31]:
!curl https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/main/Dataset/WMT.csv >> WMT.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89556  100 89556    0     0   462k      0 --:--:-- --:--:-- --:--:--  462k


In [34]:
df1 = spark.read.csv('/content/WMT.csv', header= True, inferSchema=True)

In [35]:
df1.show()

+-------------------+---------+---------+---------+---------+---------+--------+
|               Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+-------------------+---------+---------+---------+---------+---------+--------+
|2016-01-20 00:00:00|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21 00:00:00|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-22 00:00:00|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|
|2016-01-25 00:00:00|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|
|2016-01-26 00:00:00|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|
|2016-01-27 00:00:00|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|
|2016-01-28 00:00:00|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|
|2016-01-29 00:00:00|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|
|2016-02-01 00:00:00|65.910004|    67.93|65.889999|     67.5| 59.90081|14728400|
|2016-02-02 00:00:00|67.3000

In [37]:
from pyspark.sql.functions import format_number, dayofmonth, hour, dayofyear, month, year, weekofyear, date_format

In [38]:
df1.select(dayofmonth(df1['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
|               2|
|               3|
|               4|
|               5|
|               8|
|               9|
|              10|
|              11|
|              12|
|              16|
|              17|
+----------------+
only showing top 20 rows



In [39]:
df1.select(hour(df1['Date'])).show()

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [40]:
df1.select(dayofyear(df1['Date'])).show()

+---------------+
|dayofyear(Date)|
+---------------+
|             20|
|             21|
|             22|
|             25|
|             26|
|             27|
|             28|
|             29|
|             32|
|             33|
|             34|
|             35|
|             36|
|             39|
|             40|
|             41|
|             42|
|             43|
|             47|
|             48|
+---------------+
only showing top 20 rows



In [41]:
df1.select(month(df1['Date'])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
+-----------+
only showing top 20 rows



In [44]:
df1.select(month(df1['Date'])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
|          2|
+-----------+
only showing top 20 rows



In [46]:
df1.withColumn('Month', month(df1['Date'])).show()

+-------------------+---------+---------+---------+---------+---------+--------+-----+
|               Date|     Open|     High|      Low|    Close|Adj Close|  Volume|Month|
+-------------------+---------+---------+---------+---------+---------+--------+-----+
|2016-01-20 00:00:00|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|    1|
|2016-01-21 00:00:00|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|    1|
|2016-01-22 00:00:00|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|    1|
|2016-01-25 00:00:00|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|    1|
|2016-01-26 00:00:00|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|    1|
|2016-01-27 00:00:00|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|    1|
|2016-01-28 00:00:00|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|    1|
|2016-01-29 00:00:00|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|    1|
|2016-02-01 00:00:00|65.910004|    67.93|65

In [47]:
df2 = df1.withColumn('Month', month(df1['Date']))

In [49]:
df2.groupBy("Month").mean()[['avg(Month)', 'avg(Close)']].show()

+----------+------------------+
|avg(Month)|        avg(Close)|
+----------+------------------+
|      12.0|106.02932022330099|
|       1.0| 98.94980368627448|
|       6.0|  92.2302801401869|
|       3.0| 87.44880724770645|
|       5.0| 90.54859816822429|
|       9.0|100.69396066336634|
|       4.0| 91.55893247572816|
|       8.0| 96.97705391071432|
|       7.0| 96.65647596190469|
|      10.0|102.74810810810811|
|      11.0|105.59009729126215|
|       2.0| 89.16364570833336|
+----------+------------------+



In [50]:
df2.show()

+-------------------+---------+---------+---------+---------+---------+--------+-----+
|               Date|     Open|     High|      Low|    Close|Adj Close|  Volume|Month|
+-------------------+---------+---------+---------+---------+---------+--------+-----+
|2016-01-20 00:00:00|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|    1|
|2016-01-21 00:00:00|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|    1|
|2016-01-22 00:00:00|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|    1|
|2016-01-25 00:00:00|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|    1|
|2016-01-26 00:00:00|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|    1|
|2016-01-27 00:00:00|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|    1|
|2016-01-28 00:00:00|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|    1|
|2016-01-29 00:00:00|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|    1|
|2016-02-01 00:00:00|65.910004|    67.93|65

In [55]:
!curl https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/main/Dataset/customers.csv >> customers.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   207  100   207    0     0   1617      0 --:--:-- --:--:-- --:--:--  1629


In [56]:
df3 = spark.read.csv('/content/customers.csv', header=True, inferSchema=True)

In [57]:
df3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Phone: long (nullable = true)
 |-- Group: string (nullable = true)



In [58]:
df3.show()

+-------+----------+-----+
|   Name|     Phone|Group|
+-------+----------+-----+
|   John|4085552424|    A|
|   Mike|3105552738|    B|
| Cassie|4085552424|    B|
|  Laura|3105552438|    B|
|  Sarah|4085551234|    A|
|  David|3105557463|    C|
|   Zach|4085553987|    C|
|  Kiera|3105552938|    A|
|  Alexa|4085559467|    C|
|Karissa|3105553475|    A|
+-------+----------+-----+



In [59]:
from pyspark.ml.feature import StringIndexer

In [60]:
df_new= spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "b"), (5, "c")],
    ["uder_id", "catagory"]
)

In [61]:
df_new.show()

+-------+--------+
|uder_id|catagory|
+-------+--------+
|      0|       a|
|      1|       b|
|      2|       c|
|      3|       a|
|      4|       b|
|      5|       c|
+-------+--------+



In [62]:
indexer = StringIndexer(inputCol='catagory', outputCol="catagoryIndex")

In [64]:
indexed = indexer.fit(df_new).transform(df_new)

In [65]:
indexed.show()

+-------+--------+-------------+
|uder_id|catagory|catagoryIndex|
+-------+--------+-------------+
|      0|       a|          0.0|
|      1|       b|          1.0|
|      2|       c|          2.0|
|      3|       a|          0.0|
|      4|       b|          1.0|
|      5|       c|          2.0|
+-------+--------+-------------+



In [67]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [68]:
df_vector =spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.8]), 1.0)],
    ['id', 'hour', 'mobile', 'userFeatures', 'clicked']
)

In [69]:
df_vector.show()

+---+----+------+--------------+-------+
| id|hour|mobile|  userFeatures|clicked|
+---+----+------+--------------+-------+
|  0|  18|   1.0|[0.0,10.0,0.8]|    1.0|
+---+----+------+--------------+-------+



In [70]:
assembler = VectorAssembler(
    inputCols= ['hour', 'mobile', 'userFeatures'],
    outputCol= "features"
)

In [71]:
output= assembler.transform(df_vector)

In [72]:
output.show()

+---+----+------+--------------+-------+--------------------+
| id|hour|mobile|  userFeatures|clicked|            features|
+---+----+------+--------------+-------+--------------------+
|  0|  18|   1.0|[0.0,10.0,0.8]|    1.0|[18.0,1.0,0.0,10....|
+---+----+------+--------------+-------+--------------------+



In [73]:
output.select("features", "clicked").show()

+--------------------+-------+
|            features|clicked|
+--------------------+-------+
|[18.0,1.0,0.0,10....|    1.0|
+--------------------+-------+

